In [5]:
from IPython.display import YouTubeVideo

# MSE 8900 / CHE 8450 - Multiscale Modeling

Rachel B. Getman, Sapna Sarupria, Ulf D. Schiller

Clemson University

## Lecture 3.6: Coupling Molecular Dynamics and Lattice Boltzmann

1. Hybrid LB-MD Coupling
2. Boundary Conditions

### References
1. Sauro Succi, _The Lattice Boltzmann Equation for Complex States of Flowing Matter_, Oxford University Press, 2018.  
2. B. D&uuml;nweg and A. J. C. Ladd, Lattice Boltzmann Simualtions of Soft Matter, _Adv. Polym. Sci._ **221**, 89, 2009.

### From Newton to Navier-Stokes

![Route](assets/route.png)

### Fluid-Particle Coupling

* bead-spring model
* bond potential (FENE)

![Bead-spring polymer chain](assets/singlechain.png)

* elastic surface with marker points
* force coupling at the boundary

![Red blood cell](assets/lattice-mapping.png)

### Molecular Dynamics (MD)

* evolution equation for phase space vector $\Gamma$

\begin{equation*}
\Gamma(t) = e^{i\mathcal{L} t} \Gamma(0)
\end{equation*}

* Liouville operator ($\vec{F}_i$ conservative forces)

\begin{equation*}
i \mathcal{L} = \sum_i \left( \frac{\vec{p}_i}{m_i} \cdot \frac{\partial}{\partial\vec{r}_i} +  \vec{F}_i \cdot \frac{\partial}{\partial\vec{p}_i}\right) = i\mathcal{L}_{\vec{r}} + i\mathcal{L}_{\vec{p}}
\end{equation*}

* updates for positions and momenta

\begin{align*}
e^{ i\mathcal{L}_{\vec{r}} \Delta t } \vec{r}_i(t) &= \vec{r}_i(t) + \frac{\Delta t}{m_i} \vec{p}_i &
e^{ i\mathcal{L}_{\vec{p}} \Delta t } \vec{p}_i(t) &= \vec{p}_i(t) + \Delta t \vec{F}_i
\end{align*}

* Trotter formula

\begin{equation*}
e^{ i ( \mathcal{L}_1 + \mathcal{L}_2 ) t } = \lim_{n\rightarrow\infty} \left[ e^{ \frac{i\mathcal{L}_2 t}{2n} } e^{ \frac{i \mathcal{L}_1 t}{n} } e^{ \frac{i\mathcal{L}_2 t}{2n} } \right]^n
\end{equation*}

* Verlet splitting

\begin{equation*}
e^{ i(\mathcal{L}_1 + \mathcal{L}_2) \Delta t } = e^{ i\mathcal{L}_2 \frac{\Delta t}{2} } e^{ i\mathcal{L}_1 \Delta t} e^{ i\mathcal{L}_2 \frac{\Delta t}{2}} + \mathcal{O}(\Delta t^3)
\end{equation*}

### Mapping between particles and lattice Boltzmann


![Lattice Coupling](assets/coupling.png)

* interpolation scheme for velocity

\begin{equation*}\label{eq:IBM-interpolation}
\vec{u}(\vec{R}_i,t) = \mathcal{I}_a[\vec{R}_i(t)] \, \vec{u} (\vec{x},t) = \sum_{\vec{x}} \vec{u}(\vec{x},t) \delta_a(\vec{R}_i(t)-\vec{x})
\end{equation*}

* spreading of momentum transfer

\begin{equation*}\label{eq:IBM-spreading}
\vec{F}(\vec{x},t) = \mathcal{I}_a^*[\vec{R}_i(t)] \, \vec{F}_i (t) = a^{-3} \sum_{i} \vec{F}_i(t) \delta_a(\vec{R}_i(t)-\vec{x})
\end{equation*}

\begin{equation*}
-\frac{\Delta t}{a^3}\vec{F} = {\Delta\vec{j}} = \frac{\mu}{a^2h} \sum_{\vec{x}\in\text{Cell}}\sum_i \Delta f_i(\vec{x},t)\vec{c}_i
\end{equation*}

### Spatial interpolation functions

* three-dimensional discrete $\delta$ function

\begin{align*}
\sum_{\vec{x}} \delta_a(\vec{x} - \vec{R}) &= 1 , \\ %\quad \forall \vec{R} \\
\sum_{\vec{x}} \vec{x} \, \delta_a(\vec{x} - \vec{R}) &= \vec{R} ,%\quad \forall \vec{R} \\
\end{align*}

* force and torque conservation

\begin{align*}
\sum_{\vec{x}} a^3 \vec{F}(\vec{x},t) 
&= \sum_{\vec{x}} \sum_i \vec{F}_i(t) \delta_a(\vec{x} - \vec{R}_i) 
= \sum_i \vec{F}_i(t) , \\
\sum_{\vec{x}} \vec{x} \times a^3 \vec{F}(\vec{x},t)
&= \sum_{\vec{x}} \sum_i \vec{x} \times \vec{F}_i(t) \delta_a(\vec{x} - \vec{R}_i)
= \sum_i \vec{R}_i \times \vec{F}_i(t) .
\end{align*}

### Spatial interpolation functions

* three-dimensional $\delta$ product of one-dimensional functions

\begin{equation*}
\delta_a(\vec{x}) = \phi(\frac{x}{a}) \phi(\frac{y}{a}) \phi(\frac{z}{a}) ,
\end{equation*}

* three-point interpolation gives smooth $\nabla\vec{u}$

\begin{equation*}
\phi_3(r) = 
\begin{cases}
\frac{1}{3} \left( 1 + \sqrt{1 - 3r^2} \right) & 0 \le |r| \le \frac12 \\
\frac{1}{6} \left( 5 - 3|r| - \sqrt{6 |r| - 2 - 3r^2} \right) & \frac12 \le |r| \le \frac32 \\
0 & \frac32 \le |r| .
\end{cases}
\end{equation*}

* compatible with halo thickness of $1$

### Viscous coupling of particles and fluid

![Lattice Coupling](assets/coupling.png)

* Idea: treat monomers as point particles and apply Stokesian drag
\begin{equation*}
\vec{F}=-\zeta_\text{bare} \left[\vec{V}-\vec{u}(\vec{R},t)\right] + \vec{F}_\text{stoch}
\end{equation*}

* ensure momentum conservation by transferring momentum to the fluid

* add stochastic force to fulfill fluctuation-dissipation relation

### Why do all parts need to be thermalized?

* Equations of motion are stochastic differential equations
* Fokker-Planck formalism
* Kramers-Moyal expansion

\begin{eqnarray*}
%
\text{Particle, conservative}: \quad \mathcal{L}_1 &=& -\sum_i \left( \frac{\partial}{\partial\vec{r}_i}\cdot\frac{\vec{p}_i}{m_i} + \frac{\partial}{\partial \vec{p}_i}\cdot \vec{F}_i \right) \\
%
\text{Particle, Langevin}: \quad \mathcal{L}_2 &=& \sum_i \frac{\Gamma_i}{m_i}\frac{\partial}{\partial \vec{p}_i} \vec{p}_i \\
%
\text{Particle, stochastic}: \quad \mathcal{L}_3 &=& k_BT \sum_i \Gamma_i \frac{\partial^2}{\partial\vec{p}_i^2} 
%
\end{eqnarray*}

* Fluctuation-Dissipation relation

$$\left(\sum_i \mathcal{L}_i\right) \exp(-\beta \mathcal{H}) = 0$$

### Why do all parts need to be thermalized?

\begin{eqnarray*}
%
\text{Particle, conservative}: \quad \mathcal{L}_1 &=& -\sum_i \left( \frac{\partial}{\partial\vec{r}_i}\cdot\frac{\vec{p}_i}{m_i} + \frac{\partial}{\partial \vec{p}_i}\cdot \vec{F}_i \right) \\
%
\text{Fluid, conservative:} \quad \mathcal{L}_4 &=& \int d\vec{r} \left( \frac{\delta}{\delta\rho} \partial_\alpha j_\alpha + \frac{\delta}{\delta j_\alpha} \partial_\beta \Pi^\text{eq}_{\alpha\beta} \right) \\
%
\text{Fluid, viscous:} \quad \mathcal{L}_5 &=& \eta_{\alpha\beta\gamma\delta} \int d\vec{r} \frac{\delta}{\delta j_\alpha} \partial_\beta \partial_\gamma u_\delta \\
%
\text{Fluid, stochastic:} \quad \mathcal{L}_6 &=& k_BT\eta_{\alpha\beta\gamma\delta} \int d\vec{r} \frac{\delta}{\delta j_\alpha} \partial_\beta \partial_\gamma \frac{\delta}{\delta j_\delta}\\
%
\text{Particle, coupling:} \quad \mathcal{L}_7 &=& -\sum_i \zeta_i \frac{\partial}{\partial p_{i\alpha}} u_{i\alpha} \\
%
\text{Fluid, coupling:} \quad \mathcal{L}_8 &=& -\sum_i \zeta_i \int d\vec{r} \Delta(\vec{r},\vec{r}_i) \frac{\delta}{\delta j_\alpha(\vec{r})} \left( \frac{p_{i\alpha}}{m_i} - u_{i\alpha} \right) \\
%
\text{Fluid, stochastic:} \quad \mathcal{L}_9 &=& k_BT \sum_i \zeta_i \int d\vec{r} \Delta(\vec{r},\vec{r}_i) \frac{\delta}{\delta j_\alpha(\vec{r})} \int d\vec{r}' \Delta(\vec{r}',\vec{r}_i) \frac{\delta}{\delta j_\alpha(\vec{r}')} \\
%
\text{Particle, stochastic:} \quad \mathcal{L}_{10} &=& -2 k_BT \sum_i \zeta_i \frac{\partial}{\partial p_{i\alpha}} \int d\vec{r} \Delta(\vec{r},\vec{r}_i) \frac{\delta}{\delta j_\alpha(\vec{r})}
%
\end{eqnarray*}

### Coupled equations of motion

* all force based coupling methods can be unified 

\begin{align*}
\frac{\partial}{\partial t} \vec{v}_i(t) &= - \frac{1}{m_i} \left[ \zeta \left( \vec{v}_i - \vec{u}(\vec{R}_i,t) \right) - \xi_i - (1-r) \vec{F}_i^\text{int} \right] \\
\frac{\partial}{\partial t} \vec{u}(\vec{R}_i,t) &= \frac{1}{\rho a^3} \left[ \zeta \left( \vec{v}_i - \vec{u}(\vec{R}_i,t) \right) - \xi_i  + r \vec{F}_i^\text{int} \right]
\end{align*}

* second-order accurate force scheme $\alpha = \frac{h \zeta}{m_i}$, $\beta = \frac{h \zeta}{\rho a^3}$

\begin{align*}
\vec{v}_i(t+h) &= \vec{v}_i(t) -
\frac{\alpha}{1 + \frac{\alpha}{2}+\frac{\beta}{2}}
\left[ \vec{v}_i(t) - \vec{u}(\vec{R}_i,t) - \frac{1}{\zeta} \xi_i
- \frac{1-r+\frac{\beta}{2}}{\zeta} \vec{F}_i^\text{int} \right]
\\
\vec{u}(\vec{R}_i,t+h) &= \vec{u}(\vec{R}_i,t) +
\frac{\beta}{1 + \frac{\alpha}{2}+\frac{\beta}{2}}
\left[ \vec{v}_i(t) - \vec{u}(\vec{R}_i,t) - \frac{1}{\zeta} \xi_i
+ \frac{r + \frac{\alpha}{2}}{\zeta} \vec{F}_i^\text{int} \right]
\end{align*}

[UDS, Comp. Phys. Comm. 185, 2586-2597 (2014)]

### Unification of forcing schemes

* no-slip boundary condition can be satisfied by the choice

\begin{align*}
\zeta &= \frac{\rho a^3}{h} \frac{2}{1 + \frac{\rho a^3}{m_i}}
= \frac{m_i}{h} \frac{2}{1 + \frac{m_i}{\rho a^3}}
%\rightarrow 
%\begin{cases}
%2 \frac{\rho a^3}{h} & \text{for }  m_i \gg \rho a^3 \\
%2 \frac{m_i}{h}  & \text{for } m_i \ll \rho a^3
%\end{cases}
&r &= \frac{1}{1 + \frac{m_i}{\rho a^3}}
\end{align*}

* $r$ is controlled by ratio of the particle mass $m_i$ and the fluid
  mass $\rho a^3$ per unit cell of the lattice

\begin{equation*}
r = \frac{1}{1 + \frac{m_i}{\rho a^3}}
\end{equation*}

* $r$ can be called "immersion number"

\begin{equation*}
\begin{cases}
m_i \gg \rho a^3 \Rightarrow r \rightarrow 0:
& \text{external boundary force (EBF)}\\
m_i \ll \rho a^3 \Rightarrow r \rightarrow 1:
& \text{immersed boundary method (IBM)}
\end{cases}
\end{equation*}

[UDS, Comp. Phys. Comm. 185, 2586-2597 (2014)]

### "Bare" vs. effective friction constant

* the input friction $\zeta_\text{bare}$ is not the real friction

* $D_0 > k_B T / \zeta_\text{bare}$ (due to long time tail)
\begin{align*}
  \vec{V} &= \frac{1}{\zeta_\text{bare}} \vec{F} + \vec{u}_{av} \\
  \vec{u} &\approx \frac{1}{8 \pi \eta r} \left( 
      {\mathsf{I}} + \hat{r} \otimes \hat{r} \right) \vec{F} \\
  \vec{u}_{av} &= \frac{1}{g \eta a} \vec{F}
\end{align*}

\begin{equation*}
      \frac{1}{\zeta_\text{eff}} = \frac{1}{\zeta_\text{bare}}
      + \frac{1}{g \eta a}
\end{equation*}

* Stokes contribution from interpolation with range $a$

* this _regularizes_ the theory (no point particles in nature!)

* $\zeta_\text{bare}$ has no physical meaning!

### Finite size effects

Study diffusion / sedimentation of a single object

![Flowfield around sphere](assets/trivial_flowfield.png)

* $L = \infty$: $u(r) \sim 1/r$
* $F \sim \eta R v = \eta R^2 (v/R)$
* area $R^2$, shear gradient $v/R$

![Flowfield in a box](assets/boxflowfield.png)

*  $L < \infty$: homogeneous background force (no net acceleration)
*  backflow due to momentum conservation
*  additional shear gradient $v/L$
*  additional force $\eta R^2 (v/L) = \eta R v (R/L)$
*  _finite size effect_ $\sim R/L$

### Polymer chain in solution

![Polymer chain on lattice](assets/singlechain.png)

* bead-spring model
* bond potential (FENE)

$$V_\text{FENE} = - \frac{1}{2} k_\text{FENE} R_0^2 \ln \left[ 1 - \left(
    \frac{r}{R_0} \right)^2 \right]$$

* excluded volume (LJ/WCA)

$$V_{\text{LJ}} = 4 \epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left(
    \frac{\sigma}{r} \right)^6 + \frac{1}{4} \right], \quad r\leq2^{\frac{1}{6}}\sigma$$

### BD vs. LB for single polymer chain

#### Rouse modes

\begin{equation*}
\vec{X}_p=\frac{1}{N}\sum_{i=1}{N} \vec{r}_i \cos \left[ \frac{p \pi}{N} \left( i - \frac{1}{2} \right) \right]
\end{equation*}

![Rouse modes](assets/Fig6.png)

#### Dynamic structure factor

\begin{equation*}
S(k,t) = \frac{1}{N} \sum_{i,j}  \exp \left[ i \vec{k} \cdot \left( \vec{r}_i(t) - \vec{r}_j(0) \right) \right]
\end{equation*}

![Dynamic structure factor](assets/Fig10.png)

[T. Pham, UDS, J. R. Prakash, B. Duenweg, J. Chem. Phys. 131, 16114 (2009)]

### Finite size scaling


#### Center-of-mass diffusion

![Center-of-mass](assets/Fig5.png)

#### First Rouse mode $X_1(t)$

![First Rouse mode](assets/Fig7.png)

[T. Pham, UDS, J. R. Prakash, B. Duenweg, J. Chem. Phys. 131, 16114 (2009)]

### Scaling of the dynamic structure factor

* best data collapse for $z \approx 2.75$
* close to Zimm scaling

![Rouse scaling](assets/Skz3p7_labeled.png)
![Zimm scaling](assets/Skz3_labeled.png)
![Best collapse](assets/Skz2p75_labeled.png)

### Lattice representation of rigid objects

![Boundary mapping](assets/boundary_mapping.png)

* determine the points where the surface of the rigid object intersects the lattice links
* surface markers

> "Accounting for these constraints may be trivial under idealized
    conditions [...] but generally speaking, it constitutes a very
    delicate (and sometimes nerve-probing!) task."  
  Sauro Succi

### Boundary conditions

![Boundary reflections](assets/boundary-reflections.png)

* these rules are simple to implement
* but they are **only correct to first order**
* the boundary location is **always midway in between nodes**

### Interpolation boundary conditions

![Bouzidi boundaries](assets/bouzidi_boundaries.png)

\begin{equation*}
\begin{aligned}
f_{i^-}(\vec{R}_B,t+h) &= 2q f_i^*(\vec{R}_B,t) + (1-2q) f_i^*(\vec{R}_B-h\vec{c}_i,t) , &\qquad& q < \frac{1}{2} , \\
f_{i^-}(\vec{R}_B,t+h) &= \frac{1}{2q} f_i^*(\vec{R}_B,t) + \frac{2q-1}{2q} f_{i^-}^*(\vec{R}_B,t) , &&q \geq \frac{1}{2} .
\end{aligned}
\end{equation*}

[Bouzidi et al., Phys. Fluids 13, 3452 (2001)]

### Multi-reflection boundary conditions

![Multi-reflection boundaries](assets/multireflection.png)

\begin{equation*}
\begin{split}
f_{i^-}(\vec{R}_B,t+h) &= f_i^*(\vec{R}_B,t)
- \frac{1-2q-2q^2}{(1+q)^2} f_{i^-}^*(\vec{R}_B,t)
+ \frac{1-2q-2q^2}{(1+q)^2} f_i^*(\vec{R}-h\vec{c}_i,t)\\
&\qquad - \frac{q^2}{(1+q)^2} f_{i^-}^*(\vec{R}-h\vec{c}_i,t)
+ \frac{q^2}{(1+q)^2} f_i^*(\vec{R}-2h\vec{c}_i,t) .
\end{split}
\end{equation*}

* match Taylor expansion at the boundary with Chapman-Enskog result
* yields a condition for the relaxation rate of the kinetic modes

$$\lambda_g(\lambda_s)=-8\frac{2+\lambda}{8+\lambda}$$

* sometimes called "magic" although there is no magic involved

[Ginzburg and d'Humieres, Phys. Rev. E 68, 066614 (2003)]

### Equilibrium interpolation

* interpolation for equilibrium
* bounce-back for non-equilibrium
* non-equilibrium enters Chapman-Enskog one order later than equilibrium
* still second order accurate!

\begin{equation*}
\begin{aligned}
f_{i^-}^\text{eq}(\vec{R}_B,t+h) &= 2q f_i^\text{eq}(\vec{R}_B,t) + (1-2q) f_i^\text{eq}(\vec{R}_B-h\vec{c}_i,t) \qquad && q<\frac{1}{2} \\
f_{i^-}^\text{eq}(\vec{R}_B,t+h) &= \frac{1-q}{q} f_i^\text{eq}(\vec{R},t) + \frac{2q-1}{q} f_i^\text{eq}(\vec{R}_B+qh\vec{c}_i) && q \geq \frac{1}{2}  \\
f_{i^-}^\text{nq}(\vec{R}_B,t+h) &= f_i^\text{nq}(\vec{R}_B,t) 
\end{aligned}
\end{equation*}

[Chun and Ladd, Phys. Rev. E 75, 066705 (2007)]

### LBM Summary

* Lattice Boltzmann: lattice kinetic approach to hydrodynamics
* Solid theoretical underpinning 
* Coupling Molecular Dynamics and Lattice Boltzmann
* Unification of force coupling schemes
* Applications: polymers, cells, porous media
* consistent thermal fluctuations
* beyond Navier-Stokes: possible but can get complicated
* challenges: non-ideal fluids, multi-phase fluids, thermal flows

### Closing remarks

> "But, as with education in general, simulation must be kept honest,
  because seeing is believing, and animated displays can be very
  convincing irrespective of their veracity."  
    D. C. Rapaport, The Art of Molecular Dynamics Simulation
  
* A bug in the program is always more likely than discovery of new physics.
* Get the right answers for the right reasons!

### Hands-On Activity

1. Implement a coupling between your MD code and the LB code available at https://gist.github.com/uschille/8f65dd40572b2d943409.
2. Simulate a tagged particle in an LB fluid and observe the behavior of the displacement and velocity.
3. Move the particle with a constant velocity through the fluid and measure the drag force on the particle. Can you validate the Stokes-Einstein relation?
4. Choose a few positions on the lattice and repeat the measurement of the drag force when the particle is kept fixed. Does the drag force depend on the chosen position?

### References

1. B. D&uuml;nweg, and A. J. C. Ladd. Lattice Boltzmann Simulations of Soft Matter Systems. _Adv. Poly. Sci._ **221**, 89 (2008)
2. C. Aidun and J. Clausen. Lattice-Boltzmann Method for Complex Flows. _Annu. Rev. Fluid Mech._ **42**, 439-472 (2010)
3. UDS, Timm Kr&uuml;ger, and Oliver Henrich. Mesoscopic Modelling and Simulation of Soft Matter. _Soft Matter_  (2017)
4. UDS and O. Kuksenok. Lattice-Boltzmann Modeling of Multicomponent Systems: An Introduction. _Rev. Comput. Chem._ (2017)
5. S. Succi. _The Lattice Boltzmann Equation for Complex States of Flowing Matter_. Oxford University Press (2018)